<a href="https://colab.research.google.com/github/rokarlysson/chatbotmiru/blob/main/chatbotmiru.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import io
import os
import threading
import tkinter as tk
from tkinter import filedialog
from typing import Optional

import cv2
from PIL import Image, ImageTk
import google.generativeai as genai
import google.ai.generativelanguage as glm

# Substitua com sua chave de API do Google Cloud
API_KEY = os.getenv("GOOGLE_API_KEY")

if API_KEY is None:
    raise ValueError(
        "Você precisa definir a variável de ambiente GOOGLE_API_KEY com sua chave de API."
    )

genai.configure(api_key=API_KEY)
model = genai.GenerativeModel("gemini-pro-vision")

class ChatbotGUI:
    def __init__(self, master):
        self.master = master
        master.title("Chatbot com Visão")

        self.cap = None
        self.current_frame = None

        self.chat_history = tk.Text(master, state="disabled")
        self.chat_history.pack()

        self.input_entry = tk.Entry(master, width=50)
        self.input_entry.pack()

        self.send_button = tk.Button(
            master, text="Enviar", command=self.send_message
        )
        self.send_button.pack()

        self.camera_button = tk.Button(
            master, text="Usar Câmera", command=self.start_camera
        )
        self.camera_button.pack()

        self.upload_button = tk.Button(
            master, text="Carregar Imagem", command=self.upload_image
        )
        self.upload_button.pack()

        self.stop_camera_button = tk.Button(
            master, text="Parar Câmera", command=self.stop_camera, state="disabled"
        )
        self.stop_camera_button.pack()

        self.canvas = tk.Canvas(master, width=320, height=240)
        self.canvas.pack()

        self.photo = None

    def send_message(self):
        user_input = self.input_entry.get()
        self.input_entry.delete(0, tk.END)

        self.append_to_chat("Você: " + user_input)

        # TODO: integrar com WhatsApp para receber mensagens e imagens
        #  - receber input do usuário (texto ou imagem)
        #  - processar a resposta
        #  - enviar a resposta de volta ao WhatsApp

    def process_response(self, response_text: str):
        self.append_to_chat("Chatbot: " + response_text)

    def append_to_chat(self, text):
        self.chat_history.config(state="normal")
        self.chat_history.insert(tk.END, text + "\n")
        self.chat_history.config(state="disabled")
        self.chat_history.see(tk.END)

    def start_camera(self):
        self.cap = cv2.VideoCapture(0)
        self.camera_button.config(state="disabled")
        self.stop_camera_button.config(state="normal")

        self.video_stream()

    def video_stream(self):
        if self.cap.isOpened():
            ret, frame = self.cap.read()
            if ret:
                self.current_frame = frame
                cv2image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                img = Image.fromarray(cv2image)
                self.photo = ImageTk.PhotoImage(image=img)
                self.canvas.create_image(0, 0, image=self.photo, anchor=tk.NW)

            self.master.after(10, self.video_stream)

    def stop_camera(self):
        if self.cap is not None and self.cap.isOpened():
            self.cap.release()
            self.camera_button.config(state="normal")
            self.stop_camera_button.config(state="disabled")

    def upload_image(self):
        file_path = filedialog.askopenfilename(
            filetypes=[("Image files", "*.jpg *.jpeg *.png")]
        )
        if file_path:
            self.describe_image(file_path)

    def describe_image(self, image_path: str):
        with open(image_path, "rb") as image_file:
            image_bytes = image_file.read()

        self.generate_description(image_bytes)

    def generate_description(self, image_bytes: bytes):
        blob = glm.Blob(mime_type="image/jpeg", data=image_bytes)
        response = model.generate_content(["Descreva esta imagem.", blob], stream=True)

        description = ""
        for chunk in response:
            description += chunk.text

        self.process_response(description)


if __name__ == "__main__":
    root = tk.Tk()
    chatbot = ChatbotGUI(root)
    root.mainloop()


# **Funcionalidades:**

# - **Chatbot:** Permite conversas básicas com o usuário.
# - **Identificação de Imagens via Câmera:** Utiliza a webcam para capturar imagens e enviá-las para a API do Google Vision para descrição.
# - **Leitura de Imagens da Biblioteca:** Permite ao usuário selecionar uma imagem de sua biblioteca para descrição.
# - **Integração com WhatsApp (TODO):** A base para receber mensagens e imagens do WhatsApp está presente, mas a implementação da integração em si precisa ser feita.

# **Observações:**

# - **API Key:** Você precisa fornecer sua chave da API do Google Cloud.
# - **Bibliotecas:** Certifique-se de ter as bibliotecas `opencv-python`, `Pillow`, `google-generativeai`, `google-ai-generativelanguage` instaladas.
# - **WhatsApp:** A integração com o WhatsApp é um desafio à parte e exige pesquisa e desenvolvimento adicionais.

# Este código oferece uma base para construir um chatbot com funcionalidades de visão computacional. A implementação da integração com o WhatsApp e o desenvolvimento de um sistema de chatbot mais avançado ficam como próximos passos.

